In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_683848/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        # self.discard_below_average(sort_by="r2")
        # self.discard_below_average(sort_by="r2_vt")
        # self.discard_high_standard_deviation()
        self.results_df.sort_values(by="mse")
        self.results_df.to_excel(f"./results/better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer(0.8)
analize.Analize()
# analize.clean_folder(subfolder="dataset", extension="pkl")
# analize.clean_folder(subfolder="results", extension="xlsx")
# analize.clean_folder(subfolder="results", extension="txt")
# analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_sup,r2_test,r2_val,r2_vt,mse,mse_sup,mse_test,mse_val,mse_vt,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_1_7_32,0.993964,0.725931,0.991591,0.994157,0.992696,0.019860,1.832702,0.041201,0.017807,0.029504,0.308626,0.140925,1.022369,0.143180,167.838116,340.548762,"Hidden Size=[15, 2], regularizer=0.02, learnin..."
1,model_1_7_31,0.993963,0.725877,0.991795,0.994279,0.992866,0.019862,1.833063,0.040202,0.017433,0.028817,0.310270,0.140934,1.022371,0.143189,167.837868,340.548515,"Hidden Size=[15, 2], regularizer=0.02, learnin..."
2,model_1_7_33,0.993963,0.725986,0.991387,0.994033,0.992526,0.019863,1.832336,0.042201,0.018183,0.030192,0.307340,0.140935,1.022372,0.143190,167.837832,340.548478,"Hidden Size=[15, 2], regularizer=0.02, learnin..."
3,model_1_7_34,0.993960,0.726040,0.991182,0.993908,0.992354,0.019871,1.831970,0.043206,0.018564,0.030885,0.306057,0.140966,1.022382,0.143222,167.836944,340.547591,"Hidden Size=[15, 2], regularizer=0.02, learnin..."
4,model_1_7_30,0.993960,0.725821,0.991996,0.994399,0.993033,0.019872,1.833436,0.039216,0.017067,0.028142,0.311576,0.140968,1.022382,0.143224,167.836884,340.547530,"Hidden Size=[15, 2], regularizer=0.02, learnin..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
476,model_1_0_4,0.894863,0.687417,0.897535,0.837420,0.861160,0.345926,2.090245,0.352627,1.026650,0.689639,1.374996,0.588155,1.389626,0.597566,162.123061,334.833708,"Hidden Size=[15, 2], regularizer=0.02, learnin..."
477,model_1_0_3,0.893800,0.685925,0.898142,0.836231,0.860614,0.349422,2.100218,0.350538,1.034157,0.692348,1.381311,0.591119,1.393563,0.600577,162.102953,334.813599,"Hidden Size=[15, 2], regularizer=0.02, learnin..."
478,model_1_0_2,0.892707,0.684409,0.898729,0.835006,0.860039,0.353018,2.110361,0.348519,1.041894,0.695207,1.387695,0.594153,1.397614,0.603660,162.082474,334.793121,"Hidden Size=[15, 2], regularizer=0.02, learnin..."
479,model_1_0_1,0.891583,0.682866,0.899294,0.833745,0.859433,0.356716,2.120677,0.346574,1.049860,0.698217,1.394118,0.597257,1.401779,0.606814,162.061630,334.772277,"Hidden Size=[15, 2], regularizer=0.02, learnin..."
